# Flood Model Training Notebook

Train a Flood ConvLSTM Model using `usl_models` lib.

In [ ]:
%load_ext autoreload
%autoreload 2
import tensorflow as tf
import keras_tuner
from usl_models.flood_ml import constants
from usl_models.flood_ml.model import FloodModel, FloodModelParams
from usl_models.flood_ml.dataset import load_dataset_windowed, load_dataset

sim_names = ["Manhattan-config_v1/Rainfall_Data_1.txt"]

2025-04-10 14:12:27.432017: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-04-10 14:12:28.317198: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-04-10 14:12:28.317298: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-04-10 14:12:28.471743: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-04-10 14:12:28.787298: I tensorflow/core/platform/cpu_feature_guar

In [ ]:
train_dataset = load_dataset_windowed(sim_names=sim_names, batch_size=4, dataset_split='train')
validation_data = load_dataset_windowed(sim_names=sim_names, batch_size=4, dataset_split='val')

2025-04-10 14:12:44.515760: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1929] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 38364 MB memory:  -> device: 0, name: NVIDIA A100-SXM4-40GB, pci bus id: 0000:00:04.0, compute capability: 8.0


In [ ]:
# Define a dictionary of hyperparameter options.
hp_options = {
    "lstm_units": [32, 64, 128],
    "lstm_kernel_size": [3, 5],
    "lstm_dropout": [0.2, 0.3],
    "lstm_recurrent_dropout": [0.2, 0.3],
    # Optionally, additional parameters such as 'm_rainfall' or 'n_flood_maps'
    # "m_rainfall": [2, 3],
    # "n_flood_maps": [4, 6],
}

In [ ]:
hypermodel = FloodModel.get_hypermodel(**hp_options)

tuner = keras_tuner.RandomSearch(
    hypermodel,
    objective="val_loss",
    max_trials=10,
    executions_per_trial=1,
    directory="flood_tuning",
    project_name="flood_model_tuning"
)

tuner.search(train_dataset, validation_data, epochs=10)

In [3]:
model = FloodModel(FloodModelParams())
model.fit(dataset, epochs=1)

2025-04-10 14:12:53.232137: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inflood_conv_lstm/conv_lstm/conv_lstm2d/while/body/_1/flood_conv_lstm/conv_lstm/conv_lstm2d/while/dropout_7/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer
2025-04-10 14:12:57.249399: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8900


186/186 [==============================] - 145s 675ms/step - loss: 0.0081 - mean_absolute_error: 0.0102 - root_mean_squared_error: 0.0900


2025-04-10 14:15:15.359465: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 353170985354654576


In [ ]:
# Test calling the model on some data.
inputs, labels_ = next(iter(dataset))
prediction = model.call(inputs)
prediction.shape

In [ ]:
# Test calling the model for n predictions
full_dataset = load_dataset(sim_names=sim_names, batch_size=1)
inputs, labels_ = next(iter(full_dataset))
predictions = model.call_n(inputs, n=4)
predictions.shape